In [2]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import re

In [214]:
enron = pd.read_csv('./enron_labeled.csv')
enron.head()

,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,...,Cat_10_level_1,Cat_10_level_2,Cat_10_weight,Cat_11_level_1,Cat_11_level_2,Cat_11_weight,Cat_12_level_1,Cat_12_level_2,Cat_12_weight,labeled
0,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15 14:45:00,frozenset({'phillip.allen@enron.com'}),frozenset({'todd.burke@enron.com'}),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,NaN,NaN,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15 14:11:00,frozenset({'phillip.allen@enron.com'}),frozenset({'kim.bolton@enron.com'}),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,NaN,NaN,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20 17:04:51,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20 17:09:00,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09 12:30:58,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'm..tholt@e...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


### Combine email bodies and subject lines

In [215]:
enron['Complete'] = None

,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,...,Cat_10_level_2,Cat_10_weight,Cat_11_level_1,Cat_11_level_2,Cat_11_weight,Cat_12_level_1,Cat_12_level_2,Cat_12_weight,labeled,Complete
0,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15 14:45:00,frozenset({'phillip.allen@enron.com'}),frozenset({'todd.burke@enron.com'}),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,NaN,NaN,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,None
1,<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15 14:11:00,frozenset({'phillip.allen@enron.com'}),frozenset({'kim.bolton@enron.com'}),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,NaN,NaN,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,None
2,<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20 17:04:51,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,None
3,<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20 17:09:00,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,None
4,<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09 12:30:58,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'm..tholt@e...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,None


In [222]:
for i in range(0, enron.shape[0]):
    enron['Complete'][i] = str(str(enron['Subject'][i]) + " " + str(enron['content'][i]))
    
enron.head()

C:\Users\fenes\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,...,Cat_10_level_2,Cat_10_weight,Cat_11_level_1,Cat_11_level_2,Cat_11_weight,Cat_12_level_1,Cat_12_level_2,Cat_12_weight,labeled,Complete
0,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15 14:45:00,frozenset({'phillip.allen@enron.com'}),frozenset({'todd.burke@enron.com'}),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,NaN,NaN,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Re: Confidential Employee Information/Lenhart ...
1,<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15 14:11:00,frozenset({'phillip.allen@enron.com'}),frozenset({'kim.bolton@enron.com'}),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,NaN,NaN,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...
2,<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20 17:04:51,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,FW: Western Wholesale Activities - Gas & Power...
3,<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20 17:09:00,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,FW: Western Wholesale Activities - Gas & Power...
4,<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09 12:30:58,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'm..tholt@e...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,FW: Western Wholesale Activities - Gas & Power...


### Clean
This function will remove certain characters and parts of speech from the text to make clustering more accurate.

The function will return two arrays:
* The cleaned text in its String form
* The cleaned text with its tokenized words and parts-of-speech intact

In [236]:
# Load the nltk stop-words
stop_words = set(stopwords.words("english"))

def clean(text):
    
    cleaned_text_raw = '' # This will hold the cleaned text as a string, without pos tags
    cleaned_text_pos = [] # This will hold the cleaned text as an array of words + their pos tags
    
    # Tokenize the text into sentences
    # Gives an array of strings that make up the message
    lines = sent_tokenize(text)
    
    # For each line in the email, clean the line
    for line in lines:
        
        # Remove the original and forwarded email headers
        # This gets rid of only the headers, which include email addresses and names of employees.
        # The actual "original" and "forwarded" message text remains intact.
        line = re.sub(r'-----Original Message.*Subject: ', '', line)
        line = re.sub(r'-.*Forwarded by.*Subject: ', '', line)
        
        # Remove leftover email addresses - 
        # some are found inside email chains, even after the previous line removes forward headers.
        line = re.sub(r'\S*@\S*\s?', '', line)
        
        # When the dataset was released, images were replaced with "<Embedded Picture (Device Independent Bitmap)>"
        # These should be removed as well
        line = re.sub(r'<Embedded Picture .Device Independent Bitmap.>', ' ', line)
        
        # Lastly, some '/' characters confuse the part-of-speech tagger. Remove these and simply replace them with spaces
        line = re.sub(r'/', ' ', line)
        
        # Tag all the words in the sentence with part of speech
        words = nltk.pos_tag(word_tokenize(line))
        
        # Remove any stopwords from the sentence.
        words = [w for w in words if not w[0] in stop_words]
        
        # Remove any numbers from the sentence (CD)
        words = [w for w in words if not w[1] == "CD"]
        
        # Note: originally, I wanted to remove names because they don't really give context to the
        # email. However, it turns out a lot of things are labelled as proper nouns, so this removes
        # too much context from the email. I want to look into entity recognition to see if I can limit
        # this to only removing peoples' names.
        # words = [w for w in words if not w[1] == "NNP"]
        
        # Remove punctuation (:=: ,=, .=. etc... POS=posessive apostrophes)
        words = [w for w in words if not w[1] in [":", ",", ".", "(", ")", "/", "POS"]]
        
        # Finally, append the cleaned sentence to this list and move on to the next sentence
        if words == []:
            # Ignore lines that cleaning has made empty
            continue
        cleaned_text_pos.append(words)
        
    # Convert the list of tuples to a list of only words
    for line in cleaned_text_pos:
        joined = ' '.join([t[0] for t in line])
        cleaned_text_raw += (str(joined))
        cleaned_text_raw += " "
    
        
    return cleaned_text_pos, cleaned_text_raw

In [248]:
enron['content'][12]

'[IMAGE] Qwiklist(9) MyAccount Rental History Shipping List We have received L.A. Confidential(1997) on 12/31/2001 We will be processing your QwikList selections and will send the next available dvd on your QwikList. You will receive another email when it ships. Thank You! [IMAGE] L.A. Confidential(1997) Home | Customer Service | Rental Details'

In [254]:
lines = sent_tokenize(enron['Complete'][11])
cleaned_text = []

for i in range(0, len(lines)):
    words = nltk.pos_tag(word_tokenize(lines[i]))
    cleaned_text.append(words)
    
# [i[0] for i in cleaned_text[0]]
cleaned_text[len(lines) - 5]

[('10/18/01', 'CD'),
 ('-', ':'),
 ('Margaret', 'NNP'),
 ('Allen', 'NNP'),
 ('...', ':'),
 ('.needed', 'VBD'),
 ('your', 'PRP$'),
 ('advice', 'NN'),
 ('on', 'IN'),
 ('whether', 'IN'),
 ('her', 'PRP$'),
 ('sister', 'NN'),
 ('should', 'MD'),
 ('take', 'VB'),
 ('job', 'NN'),
 ('with', 'IN'),
 ('Cal-Pine', 'NNP'),
 ('or', 'CC'),
 ('Kinder', 'NNP'),
 ('Morgan', 'NNP'),
 ('2', 'CD'),
 ('.', '.')]

In [249]:
pos, raw = clean(enron['Complete'][13])
print(raw)

[ Second Delivery WPTF Friday Amen Burrito ] Sorry gang new computer messed e-mail list It 's I think I fixed Maybe Bear getting second time morning gba X-Mozilla-Status2 Message-ID Date Fri Aug -0700 From Gary Ackerman Reply-To Organization Foothill Services X-Mailer Mozilla C-UDP EBM-APPLE Macintosh U PPC MIME-Version To webmaster Subject WPTF Friday Amen Burrito Content-Type multipart alternative THE FRIDAY BURRITO `` fun fortune cookie least accurate '' Everyone getting act When I started gig I guy town writing folks like power industry California I wrote new happening important stuff This week Governor Gray Davis decided write Burrito His epistle got press mine muscling turf Not outdone PUC President Loretta Lynch released report looks every facet California power business No stone left unturned I telling enough room business us They need clear With people like Herr Hair Peace Governor I m-Not Mr.-Rogers Davis Let Do Lynch needs Friday Burrito They re-define reality week mind-numb

In [250]:
print(pos)

[[('[', 'JJ'), ('Second', 'JJ'), ('Delivery', 'NN'), ('WPTF', 'NNP'), ('Friday', 'NNP'), ('Amen', 'NNP'), ('Burrito', 'NNP'), (']', 'NNP'), ('Sorry', 'NNP'), ('gang', 'NN'), ('new', 'JJ'), ('computer', 'NN'), ('messed', 'VBD'), ('e-mail', 'JJ'), ('list', 'NN')], [('It', 'PRP'), ("'s", 'VBZ'), ('I', 'PRP'), ('think', 'VBP'), ('I', 'PRP'), ('fixed', 'VBN')], [('Maybe', 'RB')], [('Bear', 'NNP'), ('getting', 'VBG'), ('second', 'JJ'), ('time', 'NN'), ('morning', 'NN')], [('gba', 'JJ'), ('X-Mozilla-Status2', 'NN'), ('Message-ID', 'NN'), ('Date', 'NN'), ('Fri', 'NNP'), ('Aug', 'NNP'), ('-0700', 'NN'), ('From', 'IN'), ('Gary', 'JJ'), ('Ackerman', 'NNP'), ('Reply-To', 'NN'), ('Organization', 'NN'), ('Foothill', 'NNP'), ('Services', 'VBZ'), ('X-Mailer', 'NN'), ('Mozilla', 'NNP'), ('C-UDP', 'NNP'), ('EBM-APPLE', 'JJ'), ('Macintosh', 'NNP'), ('U', 'NNP'), ('PPC', 'NNP'), ('MIME-Version', 'NN'), ('To', 'TO'), ('webmaster', 'NN'), ('Subject', 'NN'), ('WPTF', 'NNP'), ('Friday', 'NNP'), ('Amen', 'NNP'